# Welcome to your first use of an LLM!

If this is your first use of Google Colab, welcome to that, too! In later weeks we will explore this in more detail. For now, just follow the instructions and let me know any questions.

First, press the `Connect` or `Reconnect` button near the top right of this window to start a new CPU box running in the Cloud.

Next click in the "cell" immediately below this one, and press Shift+Enter to execute the cell. Ollama should be downloaded and installed. The exclamation mark at the start of the line indicates that this is a shell command, not python.



In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
################################                                          45.7%

## When this completes..

The cell above should now say: `Install complete. Run "ollama" from the command line.`

Now click in the cell below and press Shift+Enter.


In [ ]:
# Start Ollama server

import subprocess

# Start the command in a background process
process = subprocess.Popen(["ollama", "serve"])

# The kernel can continue execution while the process runs in the background
print("The 'ollama serve' process is running in the background.")

# Great!

It should now say above: `The 'ollama serve' process is running in the background`.

Now press Shift+Enter in each of the cells below in turn to try out Ollama.

In [ ]:
# Download the model that we want to use - this might take a couple of mins
# The exclamation mark at the start of the line indicates that this is a shell command, not python

!ollama pull llama3.2:1b

In [ ]:
# install the Ollama python API package

!pip install ollama

In [ ]:
# import the package

import ollama

In [ ]:
# Your first message to your very own LLM running "locally" (well, on your cloud box)

message = "Hi there! This is my first message to an LLM!"

In [ ]:
# Put this message into the format expected by Ollama

messages = [{"role": "user", "content": message}]

In [ ]:
# Let's do this!!

response = ollama.chat(model="llama3.2:1b", messages=messages)
print(response['message']['content'])

In [ ]:
import os
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI
from google.colab import userdata

In [ ]:
api_key = userdata.get('OPEN_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

In [ ]:
openai = OpenAI(api_key = api_key)

In [ ]:
message = "Hello, GPT! This is my first ever message to you! Hi!"
response = openai.chat.completions.create(model="gpt-4o-mini", messages=[{"role":"user", "content":message}])
print(response.choices[0].message.content)

In [ ]:
# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [ ]:
# Define our system prompt

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [ ]:
def user_prompt_for(website):
    user_prompt = f"""You are a helpful and concise assistant. Summarize the content of the following webpage in clear, human-friendly language.

**Website Title:** {website.title}

The contents of this website are as follows. Please:
- Provide a short, structured summary in **Markdown** format.
- Focus on the **main points, arguments, or instructions**.
- Include any **news or announcements**, if applicable.
- Do **not copy text verbatim** from the website.
- Make it **easy to understand** for someone who hasn't read the article.
- Keep the summary around **200–300 words**.

---

{website.text}
"""
    return user_prompt


In [ ]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [ ]:
def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [ ]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [ ]:
website_url = input("Enter the URL of the website you want to summarize: ")
display_summary(website_url)